In [43]:
# import libraries
from dotenv import load_dotenv
from datetime import datetime
import psycopg2
import pandas as pd 
from psycopg2 import sql
import os 

In [73]:
# Load environment variables
load_dotenv()

# Retrieve credentials from environment
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully!")
    # Create a cursor
    cursor = conn.cursor()

    # Write a query
    query = """CREATE TABLE watch_exercise_data_db
            (start_time varchar(255) null,
             speed float null,
             heart_rate float null,
             elapsed_seconds bigint null,
             Date varchar(255) null,
             cadence float null,
             distance float null
             );"""
    
    cursor.execute(query)
    # Commit the changes
    conn.commit()

    cursor.close()
    conn.close()
    print("Query has been executed!")
except Exception as e:
    print(f"An error occurred: {e}")

Connected to the database successfully!
Query has been executed!


### Watch Data

In [39]:
day_summary = pd.read_csv('/Users/daviddiaz/Desktop/Main/Heart-Rate-Monitoring-Forecast/GalaxyWatchData/com.samsung.shealth.tracker.pedometer_day_summary.2024110708.csv', delimiter=',', index_col=None)

In [41]:
day_summary_filtered = day_summary[['recommendation', 'source_package_name', 'binning_data', 'create_time', 'deviceuuid', 'active_time', 'speed', 'achievement', 'healthy_step', 'datauuid', 'walk_step_count', 'run_step_count', 'day_time', 'calorie', 'distance', 'step_count', 'update_time']]

In [49]:
# Connect to the database
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully!")

    # Create a cursor
    cursor = conn.cursor()

    # Prepare the insert statement
    insert_query = sql.SQL("""
    INSERT INTO watch_summary (
        recommendation, source_package_name, binning_data, create_time, deviceuuid, 
        active_time, speed, achievement, healthy_step, datauuid, walk_step_count, 
        run_step_count, day_time, calorie, distance, step_count, update_time
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """)

    # Convert DataFrame to list of tuples
    rows_to_insert = df.to_records(index=False).tolist()

    # Execute the batch insert
    cursor.executemany(insert_query, rows_to_insert)

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    print(f"{len(rows_to_insert)} rows inserted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Connected to the database successfully!
197 rows inserted successfully.


In [34]:
day_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   recommendation       197 non-null    int64  
 1   source_info          0 non-null      float64
 2   source_package_name  197 non-null    object 
 3   binning_data         197 non-null    object 
 4   create_time          197 non-null    object 
 5   deviceuuid           197 non-null    object 
 6   modify_sh_ver        0 non-null      float64
 7   active_time          197 non-null    int64  
 8   speed                197 non-null    float64
 9   achievement          197 non-null    object 
 10  healthy_step         197 non-null    int64  
 11  create_sh_ver        0 non-null      float64
 12  datauuid             197 non-null    object 
 13  walk_step_count      197 non-null    int64  
 14  run_step_count       197 non-null    int64  
 15  day_time             197 non-null    int

In [68]:
watch_exercise_df['start_time'] = pd.to_datetime(watch_exercise_df['start_time'].copy())
watch_exercise_df['Date'] = watch_exercise_df['start_time'].dt.strftime('%Y-%m-%d')

/var/folders/m8/3d9l71wn6bs6v2c3m0gltnpr0000gn/T/ipykernel_35082/2557111439.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  watch_exercise_df['start_time'] = pd.to_datetime(watch_exercise_df['start_time'].copy())


In [69]:
watch_exercise_df

,count_type,create_sh_ver,modify_sh_ver,time_offset,decline_distance,update_time,end_time,additional,max_rpm,pkg_name,...,calorie,mean_heart_rate,live_data,min_heart_rate,min_altitude,incline_distance,auxiliary_devices,deviceuuid,exercise_type,Date
0,30001.0,NaN,NaN,-14400000,NaN,"04/26/2024, 2:10:50 PM","04/26/2024, 2:09:18 PM",NaN,NaN,com.sec.android.app.shealth,...,74.0,93.0,04ddf549-cbc9-351f-9b1a-51511470d787.live_data,87.0,NaN,NaN,NaN,xtfbSOmziO,1001,2024-04-26
1,30001.0,NaN,NaN,-14400000,NaN,"04/26/2024, 5:25:27 PM","04/26/2024, 5:23:55 PM",NaN,NaN,com.sec.android.app.shealth,...,129.0,101.0,b3ccd525-0f82-7fba-26d7-adf73ca28653.live_data,76.0,NaN,NaN,NaN,xtfbSOmziO,1001,2024-04-26
2,30001.0,NaN,NaN,-14400000,NaN,"04/26/2024, 6:11:07 PM","04/26/2024, 6:09:34 PM",NaN,NaN,com.sec.android.app.shealth,...,104.0,103.0,bc0b0c14-f135-35f7-2601-4f09f3245ede.live_data,75.0,NaN,NaN,NaN,xtfbSOmziO,1001,2024-04-26
3,30001.0,NaN,NaN,-14400000,NaN,"04/30/2024, 12:19:34 PM","04/27/2024, 6:33:30 AM",NaN,NaN,com.sec.android.app.shealth,...,940.0,134.0,3d5f56af-cbe4-a971-1693-0e60ab8d17d7.live_data,86.0,NaN,NaN,NaN,xtfbSOmziO,1002,2024-04-27
4,30001.0,NaN,NaN,-14400000,NaN,"04/27/2024, 1:57:30 PM","04/27/2024, 1:56:00 PM",NaN,NaN,com.sec.android.app.shealth,...,57.0,95.0,ac3f23d2-1cac-a929-49a6-53d1ea9aeb02.live_data,72.0,NaN,NaN,NaN,xtfbSOmziO,1001,2024-04-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,30001.0,NaN,NaN,-25200000,NaN,"10/17/2024, 12:18:44 PM","10/17/2024, 12:17:12 PM",NaN,NaN,com.sec.android.app.shealth,...,189.0,96.0,02421eab-78e9-a673-47c1-fecd0bbd5b0a.live_data,81.0,NaN,NaN,NaN,xtfbSOmziO,1001,2024-10-17
118,30001.0,NaN,NaN,-25200000,NaN,"10/17/2024, 7:35:55 PM","10/17/2024, 7:34:23 PM",NaN,NaN,com.sec.android.app.shealth,...,225.0,95.0,70c05d86-3309-4a8f-4057-9ce858520832.live_data,73.0,NaN,NaN,NaN,xtfbSOmziO,1001,2024-10-17
119,30001.0,NaN,NaN,-25200000,NaN,"10/08/2024, 8:57:24 PM","10/08/2024, 8:55:52 PM",NaN,NaN,com.sec.android.app.shealth,...,74.0,87.0,2d4b0e5a-c835-c297-a741-e78d31131a3c.live_data,68.0,NaN,NaN,NaN,xtfbSOmziO,1001,2024-10-08
120,30001.0,NaN,NaN,-25200000,NaN,"10/12/2024, 1:17:15 PM","10/12/2024, 1:15:43 PM",NaN,NaN,com.sec.android.app.shealth,...,84.0,103.0,70437ff9-67a9-9ebf-8259-85b7540a1f1a.live_data,60.0,NaN,NaN,NaN,xtfbSOmziO,1001,2024-10-12


### Map My Run Data

In [56]:
# Map My Run Data
# Define the folder containing your JSON files
input_folder = '/Users/daviddiaz/Desktop/Main/Heart-Rate-Monitoring-Forecast/MapMyRunData/parsed_json_file'  # Replace with the actual path

# Initialize an empty list to hold DataFrames
df_list = []

# Iterate through each file in the folder
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):  # Only process .json files
        # Full path to the JSON file
        file_path = os.path.join(input_folder, filename)
        
        # Read the JSON file into a DataFrame
        df = pd.read_json(file_path, lines=True)  # Specify lines=True for line-delimited JSON
        df['time'] = pd.to_datetime(df['time'], unit='ms')
        df['elapsed_seconds'] = (df['time'] - df['time'].iloc[0]).dt.total_seconds()
        df_list.append(df)  # Add the DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
map_my_runs_df = pd.concat(df_list, ignore_index=True)

In [57]:
map_my_runs_df

,time,latitude,longitude,altitude,distance,activity_sport,activity_id,lap_start_time,lap_total_time_seconds,lap_distance_meters,date,elapsed_seconds
0,2024-09-07 02:31:07.872,NaN,NaN,NaN,NaN,Running,2024-09-07T02:31:07+00:00,2024-09-07 02:31:07+00:00,2040,6431.147839,2024-09-07,0.000
1,2024-09-07 02:31:08.304,NaN,NaN,NaN,NaN,Running,2024-09-07T02:31:07+00:00,2024-09-07 02:31:07+00:00,2040,6431.147839,2024-09-07,0.432
2,2024-09-07 02:31:09.475,NaN,NaN,NaN,NaN,Running,2024-09-07T02:31:07+00:00,2024-09-07 02:31:07+00:00,2040,6431.147839,2024-09-07,1.603
3,2024-09-07 02:31:10.584,NaN,NaN,NaN,NaN,Running,2024-09-07T02:31:07+00:00,2024-09-07 02:31:07+00:00,2040,6431.147839,2024-09-07,2.712
4,2024-09-07 02:31:11.310,NaN,NaN,NaN,NaN,Running,2024-09-07T02:31:07+00:00,2024-09-07 02:31:07+00:00,2040,6431.147839,2024-09-07,3.438
...,...,...,...,...,...,...,...,...,...,...,...,...
94828,2024-09-29 00:00:34.569,37.866091,-122.300678,7.11,6491.7138,Running,2024-09-28T23:25:45+00:00,2024-09-28 23:25:45+00:00,2093,6500.977275,2024-09-29,2088.854
94829,2024-09-29 00:00:35.658,37.866097,-122.300646,7.18,6494.8334,Running,2024-09-28T23:25:45+00:00,2024-09-28 23:25:45+00:00,2093,6500.977275,2024-09-29,2089.943
94830,2024-09-29 00:00:36.567,37.866102,-122.300615,7.25,6497.9189,Running,2024-09-28T23:25:45+00:00,2024-09-28 23:25:45+00:00,2093,6500.977275,2024-09-29,2090.852
94831,2024-09-29 00:00:37.551,37.866108,-122.300583,7.33,6500.9731,Running,2024-09-28T23:25:45+00:00,2024-09-28 23:25:45+00:00,2093,6500.977275,2024-09-29,2091.836


In [51]:
map_my_runs_df['date'] = map_my_runs_df['date'].astype('str').copy()

### Polar Data

In [59]:
# Path to the folder of data
folder_path = 'PolarFlowData'
number_of_files = 0

# Create an empty list to store all running data
all_running_data = []

# Create an empty list to store all aggregated data
all_headers_data = []
# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    # Check if the file is a CSV
    if filename.endswith('.CSV'):
        print(f"Processing file: {filename}")
        number_of_files += 1
        
        try:
            # Read the first row separately (aggregated statistics)
            header = pd.read_csv(file_path, nrows=1)
            
            # Read the rest of the data, skipping the first two rows
            running = pd.read_csv(file_path, skiprows=[0, 1])
            
            # Extract Date and Start Time from header
            date_str = header['Date'].values[0]  # Assuming it's in 'Date' column
            start_time_str = header['Start time'].values[0]  # Assuming it's in 'Start time' column

            # Combine the date and start time to form a datetime object
            start_datetime_str = f"{date_str} {start_time_str}"
            start_datetime = datetime.strptime(start_datetime_str, "%d-%m-%Y %H:%M:%S")
            
            # Convert 'Time' column (hh:mm:ss format) to timedeltas
            running['Time'] = pd.to_timedelta(running['Time'])

            # Create a new column 'Real_Time' by adding 'Time' column to the start datetime
            running['Real_Time'] = running['Time'] + start_datetime

            running['Source_File'] = filename

            # Append the Headers to the list
            all_headers_data.append(header)
            
            # Append the processed DataFrame to the list
            all_running_data.append(running)

        except pd.errors.EmptyDataError:
            print(f"Error: {filename} is empty or has invalid format")
        except FileNotFoundError:
            print(f"Error: {filename} not found")
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")

print(f"\n Total Amount of Files Processes: {number_of_files}")

Processing file: David+_Diaz+_2024-02-06_14-43-30.CSV
Processing file: David+_Diaz+_2024-06-21_19-35-59.CSV
Processing file: David+_Diaz+_2024-01-09_16-27-42.CSV
Processing file: David+_Diaz+_2024-07-03_17-52-41.CSV
Processing file: David+_Diaz+_2024-07-16_20-02-21.CSV
Processing file: David+_Diaz+_2024-07-24_19-57-39.CSV
Processing file: David+_Diaz+_2024-01-11_15-59-36.CSV
Processing file: David+_Diaz+_2024-01-23_14-30-16.CSV
Processing file: David+_Diaz+_2024-01-27_17-09-41.CSV
Processing file: David+_Diaz+_2024-06-26_20-05-02.CSV
Processing file: David+_Diaz+_2024-01-01_15-35-44.CSV
Processing file: David+_Diaz+_2024-07-27_18-05-44.CSV
Processing file: David+_Diaz+_2024-01-16_14-07-56.CSV
Processing file: David+_Diaz+_2024-02-16_14-21-16.CSV
Processing file: David+_Diaz+_2024-06-03_18-49-07.CSV
Processing file: David+_Diaz+_2024-01-18_10-42-57.CSV
Processing file: David+_Diaz+_2024-06-09_17-36-33.CSV
Processing file: David+_Diaz+_2024-02-03_12-12-54.CSV
Processing file: David+_Diaz

In [60]:
# After processing all files, concatenate all the DataFrames into one
if all_running_data:
    combined_running_df = pd.concat(all_running_data, ignore_index=True)
    
else:
    print("No data to concatenate.")

In [63]:
# Rename the Columns into Name without whitespaces
combined_running_df.rename(columns={'HR (bpm)': 'HR', 'Speed (mi/h)': 'Speed', 'Pace (min/mi)': 'Original_Pace', 'Distances (ft)': 'distances'}, inplace=True)

In [64]:
combined_running_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35850 entries, 0 to 35849
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   Sample rate         24 non-null     float64        
 1   Time                35850 non-null  timedelta64[ns]
 2   HR                  22218 non-null  float64        
 3   Speed               35850 non-null  float64        
 4   Original_Pace       35850 non-null  object         
 5   Cadence             0 non-null      float64        
 6   Altitude (ft)       0 non-null      float64        
 7   Stride length (in)  0 non-null      float64        
 8   distances           35850 non-null  float64        
 9   Temperatures (F)    0 non-null      float64        
 10  Power (W)           0 non-null      float64        
 11  Unnamed: 11         0 non-null      float64        
 12  Real_Time           35850 non-null  datetime64[ns] 
 13  Source_File         35850 non-n

In [ ]:
polar_data_filtered = [['Time', 'HR', 'Speed', 'Original_Pace', 'distances', 'Real_Time', 'Source_File' ]]

In [102]:
# Extract the date part from the 'Real_Time' column
combined_running_df['Date'] = combined_running_df['Real_Time'].dt.strftime('%Y-%m-%d')

### All Trails Data

In [103]:
hikes = pd.read_csv('/Users/daviddiaz/Desktop/Main/Heart-Rate-Monitoring-Forecast/AllTrailsData/hikes_overview.csv')

## Find dates

In [122]:
hikes['Date'] = pd.to_datetime(hikes['Date'], format='%m-%d-%Y').dt.strftime('%Y-%m-%d')
hikes['Date'] = hikes['Date'].astype('str')

In [123]:
# Hikes
hikes_dates = hikes['Date'].unique()
# Polar Runs
polar_runs_dates = combined_running_df['Date'].unique()
# Map my Run Runs
map_my_runs_dates = map_my_runs_df['date'].unique()
# Watch walk dates
watch_walk_dates = watch_walks['Date'].unique()
# Watch run dates
watch_run_dates = watch_runs['Date'].unique()

In [162]:
watch_walks = watch_exercise_df[watch_exercise_df['exercise_type'] == 1001]
watch_runs = watch_exercise_df[watch_exercise_df['exercise_type'] == 1002]

In [163]:
# Convert the lists of dates to sets
hikes_dates = set(hikes['Date'].unique())
polar_runs_dates = set(combined_running_df['Date'].unique())
# Watch walk dates
watch_walk_dates = set(watch_walks['Date'].unique())
# Watch run dates
watch_run_dates = set(watch_runs['Date'].unique())

# Find matches between pairs of datasets
matches = {
    'hikes & polar_runs': hikes_dates.intersection(polar_runs_dates),
    'hikes & map_my_runs': hikes_dates.intersection(map_my_runs_dates),
    'hikes & watch_walks': hikes_dates.intersection(watch_walk_dates),
    'hikes & watch_runs': hikes_dates.intersection(watch_run_dates),
    'polar_runs & map_my_runs': polar_runs_dates.intersection(map_my_runs_dates),
    'polar_runs & watch_walks': polar_runs_dates.intersection(watch_walk_dates),
    'polar_runs & watch_runs': polar_runs_dates.intersection(watch_run_dates),
    'map_my_runs & watch_walks': map_my_runs_dates.intersection(watch_walk_dates),
    'map_my_runs & watch_runs': map_my_runs_dates.intersection(watch_run_dates)
}

print("Matching dates between datasets:", matches)

Matching dates between datasets: {'hikes & polar_runs': {'2024-07-16'}, 'hikes & map_my_runs': {'2024-11-01'}, 'hikes & watch_walks': {'2024-08-07', '2024-11-01', '2024-11-02', '2024-09-01', '2024-10-20'}, 'hikes & watch_runs': set(), 'polar_runs & map_my_runs': {'2024-06-03'}, 'polar_runs & watch_walks': set(), 'polar_runs & watch_runs': set(), 'map_my_runs & watch_walks': {'2024-05-04', '2024-11-07', '2024-04-27', '2024-08-13', '2024-10-08', '2024-10-15', '2024-10-22', '2024-10-29', '2024-09-03', '2024-10-27', '2024-10-19', '2024-10-12', '2024-11-01'}, 'map_my_runs & watch_runs': {'2024-04-30', '2024-04-27', '2024-10-29', '2024-08-13', '2024-05-09', '2024-10-22', '2024-10-15'}}
